In [ ]:
%matplotlib inline
import mne

# Load raw EEG file
raw = mne.io.read_raw_edf("../data/S001/S001R01.edf", preload=True)

# Print info (channels, sampling rate, etc.)
print(raw)
print(raw.info)
print(raw.ch_names)

# Plot
raw.plot()


In [ ]:
raw.filter(1., 40.)   # keep 1–40 Hz (standard EEG band)

psd = raw.compute_psd(method='welch', fmin=0.5, fmax=40.0)  # Compute PSD (Power Spectral Density) up to 40 Hz
print(psd)

psd_data = psd.get_data()
freqs = psd.freqs
print(psd_data.shape)
print(psd_data[0, :10])
print(freqs[:10])

psd.plot()

In [ ]:
import numpy as np

def bandpower(psd_data, freqs, fmin, fmax):
    """Compute average power in a specific frequency band."""
    mask = (freqs >= fmin) & (freqs <= fmax)
    return psd_data[:, mask].mean(axis=1)  # average across selected freq bins

# Now apply to each band
bands = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta":  (13, 30),
    "gamma": (30, 40)
}

bandpowers = {
    band: bandpower(psd_data, freqs, fmin, fmax) for band, (fmin, fmax) in bands.items()
}

for band, values in bandpowers.items():
    print(f"{band}: {values[0]:.3e}")



In [ ]:
import matplotlib.pyplot as plt

channel_idx = 0  # pick first channel for now
bands_list = list(bandpowers.keys())
values_list = [bandpowers[b][channel_idx] for b in bands_list]

plt.bar(bands_list, values_list)
plt.ylabel("Power (µV²/Hz)")
plt.title(f"Bandpower for channel {raw.ch_names[channel_idx]}")
plt.show()